# *Classification Network*

In [1]:
sep=",;,;,;"
def wrapper(learning_rate, dropout):
    from hops import hdfs
    #TODO: add network here with all the import within necessary for the code within the function
    
    acc=0.87
    hdfs.log(sep+str(acc)+sep)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2803,application_1513605045578_0031,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


# *Parallely executes and returns list of accuracies*

In [5]:
def get_accuracy(v):
    if sep in v["_c0"]:
        i = v["_c0"].find(sep)
        substr = v["_c0"][i+len(sep):]
        i = substr.find(sep)
        return [substr[:i]]
    else:
        return []

def get_all_accuracies(tensorboard_hdfs_logdir):
    from hops import hdfs
    print(tensorboard_hdfs_logdir)
    results=[]
    for i in range(4):
        path_to_log=tensorboard_hdfs_logdir+"/"
        for k in args_dict_grid.keys():
            path_to_log+=k+"="+str(args_dict_grid[k][i])+"."
        path_to_log+="log"
        print("Path to log: ")
        print(path_to_log)
        raw = spark.read.csv(path_to_log, sep="\n")
        #raw.show(10)
        #raw.count()
        r = raw.rdd.flatMap(lambda v: get_accuracy(v)).collect()
        results.extend(r)

    #print(results)
    return [float(res) for res in results]

def execute_all(args_dict):
    from hops import tflauncher

    tensorboard_hdfs_logdir = tflauncher.launch(spark, wrapper, args_dict)
    return get_all_accuracies(tensorboard_hdfs_logdir)

# *Evolutionary algorithm for hyperparameter optimization*

In [7]:
#Define dict for hyperparameters
args_dict = {'learning_rate': [0.001,0.002], 'dropout': [0.45,0.05]}
#TODO: add here evolutive algorithm
accuracies = execute_all(args_dict)
print accuracies

An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 1.0 failed 4 times, most recent failure: Lost task 1.3 in stage 1.0 (TID 9, hadoop10, executor 2): java.io.IOException: Cannot run program "/srv/hops/anaconda/anaconda/envs/MusicClassificationandGeneration/bin/python": error=13, Permission denied
	at java.lang.ProcessBuilder.start(ProcessBuilder.java:1048)
	at org.apache.spark.api.python.PythonWorkerFactory.startDaemon(PythonWorkerFactory.scala:163)
	at org.apache.spark.api.python.PythonWorkerFactory.createThroughDaemon(PythonWorkerFactory.scala:89)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:65)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:117)
	at org.apache.spark.api.python.PythonRunner.compute(PythonRDD.scala:128)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:63)
	at org.apache.spar